In [1]:
import numpy as np

#  1. Метод проекции градиента

Рассматривается задача минимизации 
$$ J(u) = ||u||^2 + <u,a>^2_{\mathbb{H}} + <u,b>_{\mathbb{H}} \rightarrow \inf_{u \in U}, \qquad ||a||_{\mathbb{H}}=1, ||b||_{\mathbb{H}}=2, <a,b>_{\mathbb{H}} = 0,$$
$$ U = \Big\{u \in \mathbb{H} \quad | \quad ||u||_{\mathbb{H}} \le 2, \quad <a,u>_{\mathbb{H}} \ge 1\Big\}.$$

Взяв в качестве начального приближения элемент  $u_0 = a$, решить эту задачу с помощью метода проекции градиента с постоянным шагом $\alpha_k \equiv \frac{1}{4}$. 

**Решение:** 
$$ u_{k+1} = a + u_k - \alpha_k J'(u_k),$$
где $J'(u) = u + 2a <u,a> + b$

In [184]:
def gradient_projection(u_0, alpha, a, b, t_0=0, t_1=1, h = 10):
    '''
    Функция возвращает минимальное значение функционала и вектор, на котором оно достигается
    '''
    # Если были поданы функции, переводим их в векторы заданного формата
    if type(a) == np.ufunc:
        a = np.vectorize(a)(np.linspace(t_0, t_1, h))
    if type(b) == np.ufunc:
        b = np.vectorize(b)(np.linspace(t_0, t_1, h))
    if type(u_0) == np.ufunc:
        u_0 = np.vectorize(u_0)(np.linspace(t_0, t_1, h))

    # Инициализация переменных
    u = u_0.astype(np.float64)
    change = np.full_like(u, 10)

    # Основной цикл программы
    while(change.any() != 0):
        J_diff = b + 2 * (u + a * sum(u * a))
        change = a - alpha * J_diff
        u += change

    return sum(u**2)+ sum(u * a)**2 + sum(u * b), u

In [185]:
a = np.array([1, 0])
b = np.array([0, 2])
alpha = 1/4

J, u = gradient_projection(a, alpha, a, b)
print('Min value:\t', J)
print('Optimum:\t', u)

Min value:	 1.0
Optimum:	 [ 1. -1.]


# 2. Метод Ньютона

Рассматривается задача минимизации 
$$ J(u) = \frac{1}{2}||u - a||^4_{\mathbb{H}} + ||u - b||^2_{\mathbb{H}} \rightarrow \inf_{u \in U}, \qquad U = \Big\{u \in \mathbb{H} \quad | \quad <a + b,u>_{\mathbb{H}} \ge 2\Big\}, \quad ||a||_{\mathbb{H}}=||b||_{\mathbb{H}}=1, <a,b>_{\mathbb{H}} = 0.$$
$$ $$

Взяв в качестве начального приближения элемент  $u_0 = 2a$, решить эту задачу с помощью метода Ньютона. 

**Решение:** 
$$ u_{k+1} = u_k + \lambda A + B$$
$$ \lambda = \frac{2 - <a + b, B + u_k>}{<a + b, A>}$$
$$A = \frac{1}{1 + ||\alpha||^2}\Big(\frac{a+b}{2} - \frac{<a+b, \alpha>}{1 + 3||\alpha||^2} \alpha\Big)$$
$$B = \frac{1}{1 + ||\alpha||^2}\Big(\frac{<J'(u_k), \alpha>}{1 + 3||\alpha||^2} \alpha - \frac{J'(u_k)}{2}\Big)$$
$$\alpha = u_k - a, \qquad J'(u_k) = 2 ||u - a||^2 (u - a) + 2 (u - b)$$

In [179]:
def Newton(u_0, a, b, t_0=0, t_1=1, h = 10):
    '''
    Функция возвращает минимальное значение функционала и вектор, на котором оно достигается
    '''
    # Если были поданы функции, переводим их в векторы заданного формата
    if type(a) == np.ufunc:
        a = np.vectorize(a)(np.linspace(t_0, t_1, h))
    if type(b) == np.ufunc:
        b = np.vectorize(b)(np.linspace(t_0, t_1, h))
    if type(u_0) == np.ufunc:
        u_0 = np.vectorize(u_0)(np.linspace(t_0, t_1, h))

    # Инициализация переменных
    u = u_0.astype(np.float64)
    change = np.full_like(u, 10)

    # Основной цикл программы
    while(change.any() != 0):
        # Считаем производную в точке u_k
        J_diff = 2 * (sum((u - a)**2) * (u - a) + (u - b))

        # Считаем вспомогательные переменные
        alpha = u - a

        A_1 = (a + b) / 2 - sum((a + b) * alpha) / (1 + 3 * sum(alpha**2)) * alpha
        A_1 /= 1 + sum(alpha**2)

        B_1 = sum(J_diff * alpha) / (1 + 3 * sum(alpha**2)) * alpha - J_diff / 2
        B_1 /= 1 + sum(alpha**2)

        # Если при lambda = 0 u принадлежит U, то это и есть решение
        if sum((a + b) * (u + B_1)) >= 2:
            change = B_1
            u += change
            continue

        # Считаем lambda
        lambda_1 = (2 - sum((a + b) * (B_1 + u))) / sum((a + b) * A_1)

        change = lambda_1 * A_1 + B_1
        u += change

    return sum((u - a)**2)**2 / 2 + sum((u - b)**2), u

In [183]:
a = np.array([1, 0])
b = np.array([0, 1])

J, u = Newton(2 * a, a, b)
print('Min value:\t', J)
print('Optimum:\t', u)

Min value:	 1.5
Optimum:	 [1. 1.]


#  3. Симплекс-метод

С помощью симплекс-метода решить каноническую задачу линейного программирования:
$$ J(u) = u_1 + u_2 - u_3 + u_5 \rightarrow \inf_{u \in U},$$
$ U = \Big\{u=(u_1, u_2, u_3, u_4, u_5)^T \ge 0, u_1 + u_4 - u_5 = 1, u_1 + u_2 + 2u_4 = 3, u_3 + u_4 = 1.\Big\}$

В качестве начальной угловой точки взять $v_0 = (3,0,1,0,2)^T$

In [107]:
c = np.array([1, 1, -1, 0, 1])
b = np.array([1, 3, 1])
A = np.array([[1, 0, 0, 1, -1], [1, 1, 0, 2, 0], [0, 0, 1, 1, 0]])

v_0 = np.array([3, 0, 1, 0, 2])

In [112]:
def simplex(A, b, c, v_0):
    '''
    Функция возвращает минимальное значение функционала и точку, в которой оно достигается
    '''
    v = v_0
    J = np.inf

    print('Initial point: ', v_0)
    print('Optimization path:')

    # Ищем базисные и свободные столбцы
    J_b = np.where((A * v).sum(axis=0) != 0)[0]
    J_A = set(range(5)) - set(J_b)
    J_A = np.array(list(J_A))

    while(True):
        # Ищем матрицы B и F 
        B = A[:,J_b]
        F = A[:,J_A]

        # Вычисляем дельты
        BF = np.linalg.inv(B) @ F
        c_b = c[J_b]
        c_A = c[J_A]
        deltas = (BF.T @ c_b[:,np.newaxis] - c_A[:,np.newaxis]).flatten()

        # Ищем положительные дельты
        pos_deltas = np.where(deltas > 0)[0]

        # Если положительных дельт нет, оптимальное решение найдено
        if len(pos_deltas) == 0:
            J = sum(v * c)
            break
        
        # Выбираем следующую дельту (положительная с последним номером)
        next_delta = pos_deltas[-1]

        # Ищем положительные гаммы
        gamma_nonzero = np.where(BF[:,next_delta] > 0)[0]

        # Если положительных гамм нет, то это луч и функционал может бесконечно убывать
        if len(gamma_nonzero) == 0:
            J = -np.inf
            break
        
        # Вычисляем тетты
        thetas = v[J_b][gamma_nonzero] / BF[:,next_delta][gamma_nonzero]

        # Переходим в следующую точку
        w = np.zeros_like(v)
        w[J_b] = v[J_b] - thetas.min() * BF[:,next_delta]
        w[J_A[next_delta]] = thetas.min()
        print('\t', w)

        # Обновляем множества базисных и свободных столбцов
        J_b_t = set(J_b)
        J_b_t.remove(J_b[thetas.argmin()])
        J_b_t.add(J_A[next_delta])
        J_b = np.array(list(J_b_t))
        J_A = set(range(5)) - set(J_b)
        J_A = np.array(list(J_A))

        v = w

    print()
    print('Result:')
    print('\tOptimal point: ', v)
    print('\tInfimum: ', J)

    return J, v

In [113]:
simplex(A, b, c, v_0)

Initial point:  [3 0 1 0 2]
Optimization path:
	 [1 0 0 1 1]
	 [0 1 0 1 0]

Result:
	Optimal point:  [0 1 0 1 0]
	Infimum:  1


(1, array([0, 1, 0, 1, 0]))